In [2]:
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from tqdm import tqdm

In [3]:
width = 64
n_cls = 100
mod = resnet50(width_per_group=width)
mod.fc = torch.nn.Linear(2048, n_cls)

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 512

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

169001984it [00:06, 27015075.31it/s]                               


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
lr = 0.0001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=lr)

In [6]:
mod = mod.cuda()
for epoch in range(10):  # loop over the dataset multiple times

    # running_loss = 0.0
    # for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
    with tqdm(trainloader, unit="batch") as tepoch:
      for inputs, labels in tepoch:
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mod(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        # running_loss += loss.item()
        tepoch.set_postfix(loss=loss.item())
    # print(f'[{epoch + 1}] loss: {running_loss / len(trainloader):.3f}')

print('Finished Training')
mod = mod.cpu()

100%|██████████| 98/98 [00:04<00:00, 20.72batch/s, loss=2.45]


Finished Training
